In [10]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import geojson
import geopandas as gpd
import pandas as pd
import numpy as np
from time import time
import leafmap.foliumap as leafmap
import supervision as sv
from garuda.od import ConfusionMatrix
from IPython.display import Markdown

In [11]:
region = 'barisal'
prediction_file = f"../data/predict/processed_labels/{region}/predictions_0.25.geojson"
print(f"Region: {region}")
print(f"Prediction file: {prediction_file}")

Region: barisal
Prediction file: ../data/predict/processed_labels/barisal/predictions_0.25.geojson


In [12]:
prediction_gdf = gpd.read_file(prediction_file) 
prediction_gdf['yolo_label'] = prediction_gdf['yolo_label'].apply(lambda x: np.array(list(map(float, x.split("_")))))
print(len(prediction_gdf))
print(prediction_gdf.crs)
prediction_gdf.head()



323
EPSG:3857


,x,y,class_name,confidence,yolo_label,geometry
0,10091524,2513552,Zigzag,0.435648,"[2.0, 0.595682, 0.582899, 0.609916, 0.583933, ...","POLYGON ((10091817.155 2513299.387, 10091860.6..."
1,10097028,2519056,Zigzag,0.417002,"[2.0, 0.528323, 0.866776, 0.528387, 0.85128, 0...","POLYGON ((10097114.673 2517934.904, 10097114.8..."
2,10094276,2521808,Zigzag,0.852314,"[2.0, 0.405524, 0.364153, 0.409346, 0.348348, ...","POLYGON ((10093987.484 2522223.398, 10093999.1..."
3,10094276,2521808,Zigzag,0.732990,"[2.0, 0.170581, 0.166031, 0.185295, 0.168233, ...","POLYGON ((10093269.15 2522829.152, 10093314.13..."
4,10094276,2524559,Zigzag,0.847060,"[2.0, 0.823017, 0.667586, 0.823285, 0.65254, 0...","POLYGON ((10095263.961 2524047.39, 10095264.78..."


In [13]:
prediction_gdf.crs.to_string()
print(prediction_gdf.head(2))

          x        y class_name  confidence  \
0  10091524  2513552     Zigzag    0.435648   
1  10097028  2519056     Zigzag    0.417002   

                                          yolo_label  \
0  [2.0, 0.595682, 0.582899, 0.609916, 0.583933, ...   
1  [2.0, 0.528323, 0.866776, 0.528387, 0.85128, 0...   

                                            geometry  
0  POLYGON ((10091817.155 2513299.387, 10091860.6...  
1  POLYGON ((10097114.673 2517934.904, 10097114.8...  


In [14]:
prediction_gdf.reset_index(inplace=True, drop=True)
print(prediction_gdf.head(2))

          x        y class_name  confidence  \
0  10091524  2513552     Zigzag    0.435648   
1  10097028  2519056     Zigzag    0.417002   

                                          yolo_label  \
0  [2.0, 0.595682, 0.582899, 0.609916, 0.583933, ...   
1  [2.0, 0.528323, 0.866776, 0.528387, 0.85128, 0...   

                                            geometry  
0  POLYGON ((10091817.155 2513299.387, 10091860.6...  
1  POLYGON ((10097114.673 2517934.904, 10097114.8...  


In [15]:
intersection_gdf = gpd.sjoin(prediction_gdf, prediction_gdf, predicate="intersects")[['index_right']].reset_index()
intersection_gdf = intersection_gdf[intersection_gdf['index'] < intersection_gdf['index_right']]
len(intersection_gdf)

9

In [16]:
def get_iou(row):
    left_polygon = prediction_gdf.iloc[int(row['index'])]['geometry']
    right_polygon = prediction_gdf.iloc[int(row['index_right'])]['geometry']
    left_area = left_polygon.area
    right_area = right_polygon.area
    iou = left_polygon.intersection(right_polygon).area / left_polygon.union(right_polygon).area
    return iou, left_area, right_area

intersection_gdf[['iou', 'left_area', 'right_area']] = intersection_gdf.apply(get_iou, axis=1, result_type="expand")
intersection_gdf = intersection_gdf[intersection_gdf['iou'] > 0.33]
len(intersection_gdf)
print(intersection_gdf.head(2))

     index  index_right       iou    left_area   right_area
23      22           23  0.369057  3182.967379  4936.785632
101     98          104  0.418913  3762.205891  2056.203122


In [17]:
drop_idx = intersection_gdf.apply(lambda x: x['index'] if x['left_area'] < x['right_area'] else x['index_right'], axis=1).astype(int).values
print(len(prediction_gdf))
prediction_gdf.drop(drop_idx, inplace=True)
print(len(prediction_gdf))
display(prediction_gdf.head(2))

323
317


,x,y,class_name,confidence,yolo_label,geometry
0,10091524,2513552,Zigzag,0.435648,"[2.0, 0.595682, 0.582899, 0.609916, 0.583933, ...","POLYGON ((10091817.155 2513299.387, 10091860.6..."
1,10097028,2519056,Zigzag,0.417002,"[2.0, 0.528323, 0.866776, 0.528387, 0.85128, 0...","POLYGON ((10097114.673 2517934.904, 10097114.8..."


In [18]:
final_kilns=prediction_gdf.to_crs(epsg=4326)
display(final_kilns.head(2))

,x,y,class_name,confidence,yolo_label,geometry
0,10091524,2513552,Zigzag,0.435648,"[2.0, 0.595682, 0.582899, 0.609916, 0.583933, ...","POLYGON ((90.65634 22.01478, 90.65673 22.01475..."
1,10097028,2519056,Zigzag,0.417002,"[2.0, 0.528323, 0.866776, 0.528387, 0.85128, 0...","POLYGON ((90.70392 22.05338, 90.70393 22.05377..."


In [19]:
import os
save_handvalidation_dir = f"../hand_validation/{region}"
os.makedirs(save_handvalidation_dir, exist_ok=True)

In [20]:
version = "v1"  
batch_size = 500
for i in range(0, len(final_kilns), batch_size):
    end = min(i+batch_size, len(final_kilns))
    final_kilns.iloc[i:i+batch_size].to_file(f"../hand_validation/{region}/{version}_potential_kiln_{i}_{end-1}.geojson", driver='GeoJSON')

GARUDA INFO     : Created 317 records
